# ICNN with VFM reg

In [1]:
import sys
sys.path.insert(0, '../')
from core import *
#config
from config import *
#CUDA
initCUDA(cuda)
#supporting files
from model2 import *
from train import *
#from train import *
from helper import *
from post_process_param import *
#from post_process_compare ismport*

import os
import logging
from datetime import datetime
import config



-----------------------------------------------------
Setting device to:  0
Test:  cuda:0
-----------------------------------------------------


-----------------------------------------------------
Setting device to:  0
Test:  cuda:0
-----------------------------------------------------



/home/feolalab/anaconda3/envs/NNEUCLID/lib/python3.9/site-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647380992/work/torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


In [2]:
#Set params:
fem_material= 'Isihara'
noise_level='high'

In [3]:
#Network params
ensemble_size = 30 #paper is 30
random_init = True
n_input = 3
n_output = 6
n_hidden = [64,64,64]
#n_hidden = [128,128,128]
#n_hidden = [64, 128, 256, 256, 128, 64]
equation="Haines-Wilson"
use_dropout = True
dropout_rate = 0.2
use_sftpSquared = True
scaling_sftpSq = 1./12
opt_method = 'adam'
epochs = 10 #Paper is 500
lr_schedule = 'cyclic'
if lr_schedule == 'multistep':
    lr = 0.1
    lr_milestones = [epochs//4,epochs//4*2,epochs//4*3]
    lr_decay = 0.1
    cycle_momentum = False
elif lr_schedule == 'cyclic':
    base_lr = 0.001
    lr = base_lr
    max_lr = 0.1
    cycle_momentum = False
    step_size_up=50
    step_size_down=50
eqb_loss_factor = 1.
reaction_loss_factor = 1.
verbose_frequency = 1

In [4]:

if 'NeoHookean' in fem_material:
    loadsteps = [10,20,30]
elif 'Ogden' in fem_material:
    loadsteps = [5,10,15,20,25,30]
elif 'GentThomas' in fem_material:
    loadsteps = [10,20,30]
elif 'ArrudaBoyce' in fem_material:
    loadsteps = [5,10,15,20,25,30,35,40,45,50]
elif 'Anisotropy'  in fem_material:
    loadsteps = [5,10,15,20,25,30,35,40]
elif 'Holzapfel' in fem_material:
    loadsteps = [5,10,15,20,25,30,35,40]
else:
    loadsteps = [10,20,30,40,50,60,70,80]

In [5]:
datasets = []
for loadstep in loadsteps:

    data_path = get_data_path(fem_dir, fem_material,
                              noise_level, loadstep)
    
    logging.info(f"Loading data from: {data_path}")

    data = loadFemData(data_path,
                       noiseLevel = additional_noise,
                       noiseType = 'displacement')

    datasets.append(data)


-----------------------------------------------------
Loading data:  ../fem-data/noise=high/NeoHookean/10/
Applying noise to displacements: 0.0


../core/loadFemData.py:43: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.ux[df.bcx!=0] = df.ux_orig[df.bcx!=0]
../core/loadFemData.py:44: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values t

-----------------------------------------------------


-----------------------------------------------------
Loading data:  ../fem-data/noise=high/NeoHookean/20/
Applying noise to displacements: 0.0
-----------------------------------------------------


-----------------------------------------------------
Loading data:  ../fem-data/noise=high/NeoHookean/30/
Applying noise to displacements: 0.0
-----------------------------------------------------



../core/loadFemData.py:43: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.ux[df.bcx!=0] = df.ux_orig[df.bcx!=0]
../core/loadFemData.py:44: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values t

In [6]:
model = ICNN3(n_input=n_input,
                n_hidden=n_hidden,
                n_output=n_output,
                use_dropout=use_dropout,
                dropout_rate=dropout_rate)

Here model 2


In [7]:
from torchinfo import summary

# Example input size: (batch_size, channels, height, width)
summary(model)

Layer (type:depth-idx)                   Param #
ICNN3                                    --
├─ModuleDict: 1-1                        --
│    └─Linear: 2-1                       256
│    └─convexLinear: 2-2                 4,096
│    └─convexLinear: 2-3                 4,096
│    └─convexLinear: 2-4                 384
├─ModuleDict: 1-2                        --
│    └─Linear: 2-5                       256
│    └─Linear: 2-6                       256
│    └─convexLinear: 2-7                 18
Total params: 9,362
Trainable params: 9,362
Non-trainable params: 0

In [8]:
model.apply(init_weights)

ICNN3(
  (layers): ModuleDict(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): convexLinear()
    (2): convexLinear()
    (3): convexLinear()
  )
  (skip_layers): ModuleDict(
    (1): Linear(in_features=3, out_features=64, bias=True)
    (2): Linear(in_features=3, out_features=64, bias=True)
    (3): convexLinear()
  )
)

In [9]:
for ensemble_iter in range(ensemble_size):
    #print('\nTraining model '+str(ensemble_iter+1)+' out of '+str(ensemble_size)+'.\n')

    model, loss_history, params = train_weak(model, datasets, fem_material, noise_level)

| epoch x/xxx |   lr    |    loss    |     eqb    |  reaction  |
| epoch x/xxx |   lr    |    loss    |     eqb    |  reaction  |


KeyboardInterrupt: 